In [11]:
import numpy as np
import os
import pandas as pd

from tflite_model_maker import model_spec
from tflite_model_maker import text_classifier
from tflite_model_maker .config import ExportFormat
from tflite_model_maker .text_classifier import AverageWordVecSpec
from tflite_model_maker .text_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

In [12]:
import tensorflow as tf

print(len(tf.config.list_physical_devices('GPU')))

0


In [13]:
label_map = {0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4:'fear'}
df = pd.read_csv('data/training.csv')
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [16]:
spec = model_spec.get('average_word_vec')
train_data = DataLoader.from_csv(filename = './data/training.csv', text_column = 'text', label_column = 'label', model_spec = spec, is_training = True)
test_data = DataLoader.from_csv(filename = './data/test.csv', text_column = 'text', label_column = 'label', model_spec = spec, is_training = False)
validation_data = DataLoader.from_csv(filename = './data/validation.csv', text_column = 'text', label_column = 'label', model_spec = spec, is_training = False)


In [17]:
import tflite_model_maker
tflite_model_maker.config.QuantizationConfig(inference_input_type=tf.float32, inference_output_type=tf.float32)

In [22]:
model = text_classifier.create(train_data, model_spec = spec, epochs = 100, validation_data=test_data, batch_size = 32)

Epoch 1/100
500/500 [==============================] - 1s 2ms/step - loss: 1.6082 - accuracy: 0.3276 - val_loss: 1.5594 - val_accuracy: 0.3488
Epoch 2/100
500/500 [==============================] - 1s 1ms/step - loss: 1.5807 - accuracy: 0.3284 - val_loss: 1.5565 - val_accuracy: 0.3488
Epoch 3/100
500/500 [==============================] - 1s 1ms/step - loss: 1.5772 - accuracy: 0.3296 - val_loss: 1.5538 - val_accuracy: 0.3488
Epoch 4/100
500/500 [==============================] - 1s 2ms/step - loss: 1.5709 - accuracy: 0.3341 - val_loss: 1.5484 - val_accuracy: 0.3488
Epoch 5/100
500/500 [==============================] - 1s 2ms/step - loss: 1.5613 - accuracy: 0.3403 - val_loss: 1.5387 - val_accuracy: 0.3604
Epoch 6/100
500/500 [==============================] - 1s 2ms/step - loss: 1.5420 - accuracy: 0.3536 - val_loss: 1.5156 - val_accuracy: 0.4012
Epoch 7/100
500/500 [==============================] - 1s 1ms/step - loss: 1.5061 - accuracy: 0.3780 - val_loss: 1.4726 - val_accuracy: 0.4315

In [23]:
loss, acc = model.evaluate(validation_data)

63/63 [==============================] - 0s 1ms/step - loss: 0.5605 - accuracy: 0.8725


In [ ]:
model.export(export_dir='label-vocab', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])